In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



In [6]:
# Root directory for low noisy output data
root_dir_output_low = f'/workspace/data/output/low_noisy/results'
root_dir_output_moderate = f'/workspace/data/output/moderate_noisy/results'
root_dir_output_high = f'/workspace/data/output/high_noisy/results'

# Directory for results (where graphs will be saved)
dir_out_results =  f'/workspace/data/output/results'

# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from pathlib import Path

# ========================= CONFIG =========================
# Ajuste os caminhos das três planilhas por métrica (SSIM e PSNR)
PATHS = {
    'SSIM': {
        5:  f'{root_dir_output_low}/ssim.xlsx',
        10:  f'{root_dir_output_moderate}/ssim.xlsx',
        15:  f'{root_dir_output_high}/ssim.xlsx',
    },
    'PSNR': {
        5:   f'{root_dir_output_low}/psnr.xlsx',
        10: f'{root_dir_output_moderate}/psnr.xlsx',
        15: f'{root_dir_output_high}/psnr.xlsx',
    }
}

# Nomes de colunas esperados em cada planilha
COLS = {
    'SSIM': dict(geo='ssim_gnlm', nlm='ssim_nlm', bm3d='ssim_bm3d'),
    'PSNR': dict(geo='psnr_gnlm', nlm='psnr_nlm', bm3d='psnr_bm3d'),
}

# Saída
OUT_DIR = Path(f'{dir_out_results}')
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_XLSX = OUT_DIR / 'max_gains_summary.xlsx'

# ========================= FUNÇÕES =========================
def detect_filename_column(df):
    candidates = [c for c in df.columns if 'file' in c.lower() or 'name' in c.lower()]
    return candidates[0] if candidates else None

def coerce_numeric(df, cols):
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    return df

def max_gain_row(df, col_geo, col_ref, filename_col=None):
    delta = df[col_geo] - df[col_ref]
    idx = int(delta.idxmax())
    row = df.loc[idx]
    return dict(
        image_index=idx + 1,  # índice 1-based
        filename=(str(row[filename_col]) if filename_col and filename_col in df.columns else ''),
        geo_value=float(row[col_geo]),
        ref_value=float(row[col_ref]),
        delta_value=float(delta.loc[idx]),
    )

def process_metric(metric_name, paths_by_sigma, cols_map):
    rows = []
    for sigma, path in paths_by_sigma.items():
        f = Path(path)
        if not f.exists():
            print(f"[WARN] Missing file for {metric_name}, sigma={sigma}: {f}")
            continue
        df = pd.read_excel(f)
        filename_col = detect_filename_column(df)
        df = coerce_numeric(df, [cols_map['geo'], cols_map['nlm'], cols_map['bm3d']])

        for comp, ref in [('GEO-NLM minus NLM', 'nlm'), ('GEO-NLM minus BM3D', 'bm3d')]:
            best = max_gain_row(df, cols_map['geo'], cols_map[ref], filename_col)
            rows.append({
                'σ': sigma,
                'Comparison': comp,
                'Image Index': best['image_index'],
                'Filename': best['filename'],
                'GEO-NLM': round(best['geo_value'], 4),
                'Reference': round(best['ref_value'], 4),
                'Δ (GEO − Ref)': round(best['delta_value'], 4)
            })
    return pd.DataFrame(rows)

# ========================= EXECUÇÃO =========================
print("[INFO] Processing maximum gains for SSIM and PSNR...")

df_ssim = process_metric('SSIM', PATHS['SSIM'], COLS['SSIM'])
df_psnr = process_metric('PSNR', PATHS['PSNR'], COLS['PSNR'])

# Ordena por sigma e comparação
order = ['GEO-NLM minus NLM', 'GEO-NLM minus BM3D']
df_ssim['Comparison'] = pd.Categorical(df_ssim['Comparison'], order, ordered=True)
df_psnr['Comparison'] = pd.Categorical(df_psnr['Comparison'], order, ordered=True)
df_ssim = df_ssim.sort_values(['σ','Comparison']).reset_index(drop=True)
df_psnr = df_psnr.sort_values(['σ','Comparison']).reset_index(drop=True)

# Salva em um único XLSX (duas abas)
with pd.ExcelWriter(OUT_XLSX, engine='openpyxl') as writer:
    df_ssim.to_excel(writer, sheet_name='SSIM', index=False)
    df_psnr.to_excel(writer, sheet_name='PSNR', index=False)

print(f"[OK] Excel saved to: {OUT_XLSX}")


[INFO] Processing maximum gains for SSIM and PSNR...
[OK] Excel saved to: /workspace/data/output/results/max_gains_summary.xlsx
